In [21]:
from __future__ import annotations

In [32]:
from typing import List, Union
import math

In [85]:
class ToleratedValue:
    # represents value +- safe_range
    def __init__(
        self,
        value: float,
        safe_range: float,
    ):
        self.value = value
        self.safe_range = abs(safe_range)
        
    def __add__(self, other: Union[ToleratedValue, float, int]) -> ToleratedValue:
        if isinstance(other, ToleratedValue):
            return ToleratedValue(self.value + other.value, self.safe_range + other.safe_range)
        if isinstance(other, (float, int)):
            return ToleratedValue(self.value + other, self.safe_range)
        raise TypeError("amogla")
    
    def __sub__(self, other: Union[ToleratedValue, float, int]) -> ToleratedValue:
        if isinstance(other, ToleratedValue):
            return ToleratedValue(self.value - other.value, self.safe_range + other.safe_range)
        if isinstance(other, (float, int)):
            return ToleratedValue(self.value - other, self.safe_range)
        raise TypeError("amogla")

    def __mul__(self, other: Union[ToleratedValue, float, int]) -> ToleratedValue:
        if isinstance(other, ToleratedValue):
            rel_safe_range = (self.safe_range / self.value) + (other.safe_range / other.value)
            val = self.value * other.value
            return ToleratedValue(val, val * rel_safe_range)
        if isinstance(other, (float, int)):
            return ToleratedValue(self.value * other, self.safe_range * other)
        raise TypeError("amogla")

    def __truediv__(self, other: Union[ToleratedValue, float, int]) -> ToleratedValue:
        if isinstance(other, ToleratedValue):
            rel_safe_range = (self.safe_range / self.value) + (other.safe_range / other.value)
            val = self.value / other.value
            return ToleratedValue(val, val * rel_safe_range)
        if isinstance(other, (float, int)):
            return ToleratedValue(self.value / other, self.safe_range / other)
        raise TypeError("amogla")
    
    def __repr__(self) -> str:
        return f"{self.value:.6}±{self.safe_range/self.value*100:.3}%"
    
    def first_moment(self) -> float:
        start = self.value - self.safe_range
        end = self.value + self.safe_range

        return 1/3 * (end ** 3 - start ** 3) / (2 * self.safe_range)

In [86]:
def make_series(values, prec):
    return [ToleratedValue(v, v * prec) for v in values]

In [87]:
e3   = make_series([1.0, 2.2, 4.7], 0.4)
e6   = make_series([1.0, 1.5, 2.2, 3.3, 4.7, 6.8], 0.2)
e12  = make_series([1.0, 1.2, 1.5, 1.8, 2.2, 2.7, 3.3, 3.9, 4.7, 5.6, 6.8, 8.2], 0.1)
e24  = make_series([1.0, 1.1, 1.2, 1.3, 1.5, 1.6, 1.8, 2.0, 2.2, 2.4, 2.7, 3.0, 3.3, 3.6, 3.9, 4.3, 4.7, 5.1, 5.6, 6.2, 6.8, 7.5, 8.2, 9.1], 0.05)
e48  = make_series([1.00, 1.05, 1.10, 1.15, 1.21, 1.27, 1.33, 1.40, 1.47, 1.54, 1.62, 1.69, 1.78, 1.87, 1.96, 2.05, 2.15, 2.26, 2.37, 2.49, 2.61, 2.74, 2.87, 3.01, 3.16, 3.32, 3.48, 3.65, 3.83, 4.02, 4.22, 4.42, 4.64, 4.87, 5.11, 5.36, 5.62, 5.90, 6.19, 6.49, 6.81, 7.15, 7.50, 7.87, 8.25, 8.66, 9.09, 9.53], 0.02)
e96  = make_series([1.00, 1.02, 1.05, 1.07, 1.10, 1.13, 1.15, 1.18, 1.21, 1.24, 1.27, 1.30, 1.33, 1.37, 1.40, 1.43, 1.47, 1.50, 1.54, 1.58, 1.62, 1.65, 1.69, 1.74, 1.78, 1.82, 1.87, 1.91, 1.96, 2.00, 2.05, 2.10, 2.15, 2.21, 2.26, 2.32, 2.37, 2.43, 2.49, 2.55, 2.61, 2.67, 2.74, 2.80, 2.87, 2.94, 3.01, 3.09, 3.16, 3.24, 3.32, 3.40, 3.48, 3.57, 3.65, 3.74, 3.83, 3.92, 4.02, 4.12, 4.22, 4.32, 4.42, 4.53, 4.64, 4.75, 4.87, 4.99, 5.11, 5.23, 5.36, 5.49, 5.62, 5.76, 5.90, 6.04, 6.19, 6.34, 6.49, 6.65, 6.81, 6.98, 7.15, 7.32, 7.50, 7.68, 7.87, 8.06, 8.25, 8.45, 8.66, 8.87, 9.09, 9.31, 9.53, 9.76], 0.01)
e192 = make_series([1.00, 1.01, 1.02, 1.04, 1.05, 1.06, 1.07, 1.09, 1.10, 1.11, 1.13, 1.14, 1.15, 1.17, 1.18, 1.20, 1.21, 1.23, 1.24, 1.26, 1.27, 1.29, 1.30, 1.32, 1.33, 1.35, 1.37, 1.38, 1.40, 1.42, 1.43, 1.45, 1.47, 1.49, 1.50, 1.52, 1.54, 1.56, 1.58, 1.60, 1.62, 1.64, 1.65, 1.67, 1.69, 1.72, 1.74, 1.76, 1.78, 1.80, 1.82, 1.84, 1.87, 1.89, 1.91, 1.93, 1.96, 1.98, 2.00, 2.03, 2.05, 2.08, 2.10, 2.13, 2.15, 2.18, 2.21, 2.23, 2.26, 2.29, 2.32, 2.34, 2.37, 2.40, 2.43, 2.46, 2.49, 2.52, 2.55, 2.58, 2.61, 2.64, 2.67, 2.71, 2.74, 2.77, 2.80, 2.84, 2.87, 2.91, 2.94, 2.98, 3.01, 3.05, 3.09, 3.12, 3.16, 3.20, 3.24, 3.28, 3.32, 3.36, 3.40, 3.44, 3.48, 3.52, 3.57, 3.61, 3.65, 3.70, 3.74, 3.79, 3.83, 3.88, 3.92, 3.97, 4.02, 4.07, 4.12, 4.17, 4.22, 4.27, 4.32, 4.37, 4.42, 4.48, 4.53, 4.59, 4.64, 4.70, 4.75, 4.81, 4.87, 4.93, 4.99, 5.05, 5.11, 5.17, 5.23, 5.30, 5.36, 5.42, 5.49, 5.56, 5.62, 5.69, 5.76, 5.83, 5.90, 5.97, 6.04, 6.12, 6.19, 6.26, 6.34, 6.42, 6.49, 6.57, 6.65, 6.73, 6.81, 6.90, 6.98, 7.06, 7.15, 7.23, 7.32, 7.41, 7.50, 7.59, 7.68, 7.77, 7.87, 7.96, 8.06, 8.16, 8.25, 8.35, 8.45, 8.56, 8.66, 8.76, 8.87, 8.98, 9.09, 9.20, 9.31, 9.42, 9.53, 9.65, 9.76, 9.88], 0.005)

In [88]:
def extend_series(series, start, end):
    start_magn = math.floor(math.log10(start))
    end_magn = math.ceil(math.log10(end))
    extended = [val * 10**n for n in range(start_magn, end_magn + 1) for val in series]
    extended = [v for v in extended if start <= v.value <= end]
    return extended

In [89]:
def find_best_pair(f, series_a, series_b):
    results = []
    for a in series_a:
        for b in series_b:
            v = f(a, b)
            results.append((v, a, b))
    results.sort(key=lambda x: x[0].first_moment())

    for i in range(min(len(results), 10)):
        v, a, b = results[i]
        print(f"{i:3}: a = {a!r}, b = {b!r} => f = {v!r}")

In [107]:
vref = 1.25
vout = 3.3
opt = lambda a, b: a - 86.66

find_best_pair(opt, extend_series(e96, 50, 200), extend_series(e96, 10, 10**6))

  0: a = 86.6±1.0%, b = 10.0±1.0% => f = -0.06±-1.44e+03%
  1: a = 86.6±1.0%, b = 10.2±1.0% => f = -0.06±-1.44e+03%
  2: a = 86.6±1.0%, b = 10.5±1.0% => f = -0.06±-1.44e+03%
  3: a = 86.6±1.0%, b = 10.7±1.0% => f = -0.06±-1.44e+03%
  4: a = 86.6±1.0%, b = 11.0±1.0% => f = -0.06±-1.44e+03%
  5: a = 86.6±1.0%, b = 11.3±1.0% => f = -0.06±-1.44e+03%
  6: a = 86.6±1.0%, b = 11.5±1.0% => f = -0.06±-1.44e+03%
  7: a = 86.6±1.0%, b = 11.8±1.0% => f = -0.06±-1.44e+03%
  8: a = 86.6±1.0%, b = 12.1±1.0% => f = -0.06±-1.44e+03%
  9: a = 86.6±1.0%, b = 12.4±1.0% => f = -0.06±-1.44e+03%


In [105]:
a = ToleratedValue(115, 115*0.01)
b = ToleratedValue(69.8, 69.8 * 0.01)
